# Part 4: Deep Dive - Peer Review 시뮬레이션 (AgentReview)

**SNU AI Psychology Workshop - February 2026**

---

## 학습 목표
1. AgentReview의 멀티 에이전트 구조 이해
2. 본인 논문으로 AI 리뷰 받기
3. 리뷰어 페르소나/평가 기준 커스텀

---
## Setup

In [ ]:
# Cell 1: 환경 설정 (Colab/로컬 자동 감지)
import os
import sys

# Colab 환경 감지
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/aiworkshop_Feb2026/"
    os.makedirs(WORKSHOP_DIR, exist_ok=True)
    os.chdir(WORKSHOP_DIR)
    IN_COLAB = True
    print("🌐 Colab 환경에서 실행 중")
except ImportError:
    # 로컬 환경
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("💻 로컬 환경에서 실행 중")

print(f"작업 폴더: {WORKSHOP_DIR}")

In [ ]:
# Cell 2: 패키지 설치 + 저장소 클론
# Colab: 자동 설치
# 로컬: pyproject.toml로 미리 설치 필요 (uv pip install -e . 또는 pip install -e .)

if IN_COLAB:
    print("📦 패키지 설치 중...")
    %pip install google-generativeai python-dotenv PyPDF2 -q
    !git clone https://github.com/ahren09/agentreview.git 2>/dev/null || echo "  ✓ AgentReview already cloned"
    print("✅ 설치 완료!")
else:
    print("✅ 로컬 환경: pyproject.toml로 설치된 패키지 사용")
    print("   (미설치 시: uv pip install -e . 또는 pip install -e .)")
    
    # 로컬에서 AgentReview 클론
    import subprocess
    if not os.path.exists("agentreview"):
        print("   📥 AgentReview 클론 중...")
        subprocess.run(["git", "clone", "https://github.com/ahren09/agentreview.git"], capture_output=True)
    else:
        print("   ✓ AgentReview already exists")

In [ ]:
# Cell 3: API Key 로딩
import os

if IN_COLAB:
    try:
        from google.colab import userdata
        GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
        print("✅ Colab Secrets에서 API Key 로딩 완료")
    except:
        GEMINI_API_KEY = None
        print("⚠️ Colab Secrets에 API KEY를 추가하세요")
else:
    try:
        from dotenv import load_dotenv
        load_dotenv()
        GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
        if GEMINI_API_KEY:
            print("✅ .env 파일에서 API Key 로딩 완료")
        else:
            print("⚠️ .env 파일에 API KEY를 추가하세요")
    except ImportError:
        GEMINI_API_KEY = None

os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY or ''

print(f"\n🔑 Gemini: {'설정됨' if GEMINI_API_KEY else '없음 (리뷰 기능 제한)'}")
print("💡 논문 리뷰 기능은 LLM API key가 필요합니다")

---
## 1. 파악하기: AgentReview 코드 구조

### AgentReview 구조 (EMNLP 2024)

```
agentreview/
├── agentreview/
│   ├── arena.py          # 메인 시뮬레이션 루프 ⭐
│   ├── paper.py          # 논문 파싱 ⭐
│   ├── reviewer.py       # 리뷰어 에이전트 ⭐
│   └── prompts/          # 리뷰어 페르소나
├── data/
│   └── iclr_reviews/     # 실제 ICLR 리뷰 데이터
└── run_review.py         # 실행 스크립트
```

**핵심 흐름:**
1. 논문 PDF → 섹션별 파싱 (`paper.py`)
2. 리뷰어 에이전트 생성 - 다양한 페르소나 (`reviewer.py`)
3. 각 리뷰어가 독립적으로 평가 (`arena.py`)
4. 점수 + 코멘트 집계

In [ ]:
# Cell 6: AgentReview 폴더 구조 확인
import os

agentreview_path = "agentreview"
if os.path.exists(agentreview_path):
    print("📁 AgentReview 구조:")
    for root, dirs, files in os.walk(agentreview_path):
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        
        level = root.replace(agentreview_path, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files[:5]:
            if file.endswith('.py') or file.endswith('.txt'):
                print(f"{subindent}{file}")
else:
    print("AgentReview가 클론되지 않았습니다. Cell 2를 실행하세요.")

In [ ]:
# Cell 7: 핵심 모듈 직접 구현 (AgentReview 스타일)
import google.generativeai as genai
from dataclasses import dataclass
from typing import List, Dict

@dataclass
class ReviewerPersona:
    """리뷰어 페르소나 정의"""
    name: str
    expertise: str
    style: str  # 엄격한, 건설적, 세부적
    focus: List[str]  # 평가 중점 영역

class PaperReviewer:
    """논문 리뷰어 에이전트"""
    
    def __init__(self, persona: ReviewerPersona):
        self.persona = persona
        if GEMINI_API_KEY:
            genai.configure(api_key=GEMINI_API_KEY)
            self.model = genai.GenerativeModel('gemini-1.5-flash')
        else:
            self.model = None
    
    def review(self, paper_text: str) -> Dict:
        """논문 리뷰 수행"""
        if not self.model:
            return {"error": "API key 필요"}
        
        prompt = self._build_review_prompt(paper_text)
        
        response = self.model.generate_content(prompt)
        
        return self._parse_review(response.text)
    
    def _build_review_prompt(self, paper_text: str) -> str:
        return f"""You are {self.persona.name}, an expert reviewer in {self.persona.expertise}.
Your reviewing style is: {self.persona.style}
You focus on: {', '.join(self.persona.focus)}

Review the following paper and provide:
1. Overall Score (1-10)
2. Summary (2-3 sentences)
3. Strengths (bullet points)
4. Weaknesses (bullet points)
5. Questions for Authors
6. Recommendation (Accept/Weak Accept/Weak Reject/Reject)

PAPER:
{paper_text[:8000]}  # 토큰 제한

Provide your review in a structured format."""
    
    def _parse_review(self, review_text: str) -> Dict:
        """리뷰 텍스트 파싱 (간단 버전)"""
        return {
            "reviewer": self.persona.name,
            "expertise": self.persona.expertise,
            "review": review_text
        }

class ReviewArena:
    """멀티 리뷰어 시뮬레이션"""
    
    def __init__(self, reviewers: List[PaperReviewer]):
        self.reviewers = reviewers
    
    def run_review(self, paper_text: str) -> List[Dict]:
        """모든 리뷰어의 리뷰 수집"""
        reviews = []
        for reviewer in self.reviewers:
            print(f"  🔍 {reviewer.persona.name} 리뷰 중...")
            review = reviewer.review(paper_text)
            reviews.append(review)
        return reviews

print("✅ 클래스 정의 완료 (Gemini 사용):")
print("   - ReviewerPersona: 리뷰어 페르소나")
print("   - PaperReviewer: 개별 리뷰어 에이전트")
print("   - ReviewArena: 멀티 리뷰어 시뮬레이션")

### 기본 리뷰어 페르소나

AgentReview는 다양한 리뷰어 페르소나를 시뮬레이션합니다:

| 페르소나 | 전문 분야 | 스타일 | 중점 |
|----------|-----------|--------|------|
| Senior ML | 기계학습 | 엄격 | 방법론, 실험 |
| Cognitive Sci | 인지과학 | 건설적 | 이론, 해석 |
| Industry | 산업 응용 | 실용적 | 적용 가능성 |

### 질문

멀티 에이전트 리뷰의 장단점은?
- 장점: 다양한 관점, 편향 감소
- 단점: 비용, 일관성 문제

어떤 페르소나가 우리 연구에 유용할까요?

---
## 2. 써보기: AI 리뷰 받기

### 샘플 데이터

테스트할 논문 (AI 심리학 관련 초록 + 방법론)

In [ ]:
# Cell 10: 샘플 논문 정의
sample_paper = """
TITLE: Do Large Language Models Exhibit Human-like Cognitive Biases?

ABSTRACT:
Large language models (LLMs) have shown remarkable capabilities in various 
cognitive tasks. This paper investigates whether LLMs exhibit human-like 
cognitive biases, specifically focusing on anchoring effects and confirmation 
bias. We design a series of experiments adapted from classical psychology 
studies and evaluate GPT-4 and Claude on these tasks.

METHODS:
We adapted 5 classical cognitive bias experiments from the psychology literature:
1. Anchoring effect (Tversky & Kahneman, 1974)
2. Confirmation bias (Wason, 1960)
3. Framing effect (Kahneman & Tversky, 1981)
4. Availability heuristic (Tversky & Kahneman, 1973)
5. Hindsight bias (Fischhoff, 1975)

Each experiment was administered to GPT-4 and Claude-3 with 100 variations.
We measured bias magnitude using effect size (Cohen's d) and compared to
human baseline data from original studies.

RESULTS:
- Anchoring effect: d=0.8 (GPT-4), d=0.6 (Claude) vs d=1.2 (human)
- Confirmation bias: Present in both models, magnitude similar to humans
- Framing effect: Weaker than humans (d=0.3 vs d=0.9)

DISCUSSION:
Our findings suggest LLMs exhibit cognitive biases but with different magnitudes.
This has implications for using LLMs as cognitive models and for AI safety.
Limitations include prompt sensitivity and lack of mechanistic explanation.
"""

print("샘플 논문 준비 완료")
print(f"길이: {len(sample_paper)} 문자")

In [ ]:
# Cell 11: 리뷰어 생성 및 리뷰 실행

# 3명의 리뷰어 페르소나 정의
personas = [
    ReviewerPersona(
        name="Dr. ML Expert",
        expertise="Machine Learning and NLP",
        style="rigorous and methodological",
        focus=["experimental design", "baselines", "reproducibility"]
    ),
    ReviewerPersona(
        name="Dr. Cognitive Scientist",
        expertise="Cognitive Psychology and Human Cognition",
        style="constructive and theory-focused",
        focus=["theoretical grounding", "ecological validity", "interpretation"]
    ),
    ReviewerPersona(
        name="Dr. AI Safety Researcher",
        expertise="AI Alignment and Safety",
        style="critical and forward-looking",
        focus=["implications", "risks", "practical applications"]
    )
]

# 리뷰어 생성
reviewers = [PaperReviewer(p) for p in personas]
arena = ReviewArena(reviewers)

# 리뷰 실행
if GEMINI_API_KEY:
    print("📝 리뷰 시작...\n")
    reviews = arena.run_review(sample_paper)
    
    print("\n" + "="*60)
    print("REVIEW RESULTS")
    print("="*60)
    
    for review in reviews:
        print(f"\n👤 {review['reviewer']} ({review['expertise']})")
        print("-" * 40)
        print(review['review'][:1500])  # 처음 1500자만
        print("...")
else:
    print("Gemini API key 없음 - 리뷰 기능 사용 불가")

### DIY: 본인 논문으로 리뷰 받기

In [ ]:
# DIY Cell 13: 본인 논문 입력
# 힌트: 초록 + 방법론 + 결과 요약을 붙여넣으세요

my_paper = """
TITLE: [논문 제목]

ABSTRACT:
[초록을 여기에 붙여넣으세요]

METHODS:
[방법론 요약]

RESULTS:
[주요 결과]

DISCUSSION:
[논의 및 한계점]
"""

print(f"입력된 논문 길이: {len(my_paper)} 문자")

In [ ]:
# DIY Cell 14: 리뷰 받기
if OPENAI_API_KEY and len(my_paper) > 200:
    print("📝 본인 논문 리뷰 시작...\n")
    my_reviews = arena.run_review(my_paper)
    
    print("\n" + "="*60)
    print("MY PAPER REVIEWS")
    print("="*60)
    
    for review in my_reviews:
        print(f"\n👤 {review['reviewer']}")
        print("-" * 40)
        print(review['review'])
else:
    print("논문 내용을 입력해주세요 (Cell 13).")

---
## 3. 바꿔보기: 리뷰어 페르소나 커스텀

### 수정 포인트

| 항목 | 기본값 | 커스텀 아이디어 |
|------|--------|----------------|
| 전문 분야 | ML, CogSci | 임상심리, 신경과학 |
| 리뷰 스타일 | 일반 학술 | 특정 저널 스타일 |
| 평가 기준 | novelty, methodology | 임상 적용성, 윤리 |
| 언어 | 영어 | 한글 |

In [ ]:
# Cell 16: 심리학 저널용 리뷰어 (Before/After)

# Before: 일반 ML 리뷰어
ml_reviewer = ReviewerPersona(
    name="Dr. ML Expert",
    expertise="Machine Learning",
    style="rigorous",
    focus=["methodology", "baselines"]
)

# After: 심리학 저널 리뷰어
psych_reviewer = ReviewerPersona(
    name="Dr. Clinical Psychologist",
    expertise="Clinical Psychology and Psychotherapy Research",
    style="constructive and clinically-oriented",
    focus=[
        "clinical relevance",
        "ecological validity",
        "ethical considerations",
        "sample representativeness",
        "effect size interpretation"
    ]
)

# 신경과학 리뷰어
neuro_reviewer = ReviewerPersona(
    name="Dr. Cognitive Neuroscientist",
    expertise="Cognitive Neuroscience and Brain Imaging",
    style="detail-oriented and mechanistic",
    focus=[
        "neural mechanisms",
        "brain-behavior relationships",
        "measurement validity",
        "replication concerns"
    ]
)

print("심리학 분야 리뷰어 페르소나 정의 완료")

In [ ]:
# DIY Cell 17: 본인만의 리뷰어 페르소나 만들기

my_reviewer = ReviewerPersona(
    name="Dr. [Your Advisor's Style]",  # <- 수정
    expertise="[분야]",  # <- 수정
    style="[스타일]",  # <- 수정 (예: "supportive but thorough")
    focus=[
        # <- 평가 중점 항목 추가
        "theoretical contribution",
        "practical implications",
    ]
)

# 테스트
if OPENAI_API_KEY:
    custom_reviewer = PaperReviewer(my_reviewer)
    review = custom_reviewer.review(sample_paper)
    print(f"👤 {review['reviewer']}의 리뷰:")
    print(review['review'][:1000])

In [ ]:
# Cell 18: 한글 리뷰 생성

class KoreanReviewer(PaperReviewer):
    """한글 리뷰를 생성하는 리뷰어"""
    
    def _build_review_prompt(self, paper_text: str) -> str:
        return f"""당신은 {self.persona.name}입니다. {self.persona.expertise} 분야 전문가입니다.
리뷰 스타일: {self.persona.style}
평가 중점: {', '.join(self.persona.focus)}

다음 논문을 리뷰하고 아래 형식으로 작성해주세요:

1. 종합 점수 (1-10점)
2. 요약 (2-3문장)
3. 강점 (글머리 기호)
4. 약점 (글머리 기호)
5. 저자에게 질문
6. 판정 (게재가/수정후게재/수정후재심/게재불가)

논문:
{paper_text[:8000]}

한글로 상세하게 리뷰해주세요."""

# 한글 리뷰어 테스트
korean_persona = ReviewerPersona(
    name="김심리 교수",
    expertise="인지심리학, AI 심리학",
    style="건설적이고 교육적인",
    focus=["이론적 기여", "방법론적 엄밀성", "실용적 함의"]
)

if OPENAI_API_KEY:
    korean_reviewer = KoreanReviewer(korean_persona)
    korean_review = korean_reviewer.review(sample_paper)
    print("=== 한글 리뷰 ===")
    print(korean_review['review'])
else:
    print("API key 필요")

In [ ]:
# DIY Cell 19: 체크리스트 형식 리뷰

CHECKLIST_PROMPT = """
다음 체크리스트로 논문을 평가해주세요:

## 연구 설계 (각 항목 O/X/△)
- [ ] 연구 질문이 명확한가?
- [ ] 가설이 검증 가능한가?
- [ ] 샘플 크기가 적절한가?
- [ ] 통제 조건이 있는가?

## 방법론
- [ ] 측정 도구가 타당한가?
- [ ] 분석 방법이 적절한가?
- [ ] 재현 가능한가?

## 결과 및 해석
- [ ] 결과가 가설을 지지하는가?
- [ ] 효과 크기가 보고되었는가?
- [ ] 한계점이 논의되었는가?

## 종합
- 강점 3가지:
- 개선점 3가지:
- 권고: 
"""

# TODO: 체크리스트 형식 리뷰어 구현
# class ChecklistReviewer(PaperReviewer):
#     def _build_review_prompt(self, paper_text):
#         return CHECKLIST_PROMPT + f"\n\n논문:\n{paper_text}"

In [ ]:
# Cell 20: 리뷰어별 결과 비교
print("""
📊 리뷰어 페르소나별 특징

| 페르소나 | 장점 | 단점 | 추천 용도 |
|----------|------|------|----------|
| ML Expert | 기술적 정확성 | 이론 관심 낮음 | AI 논문 |
| Cognitive Sci | 이론 연결 | 구현 관심 낮음 | 인지과학 논문 |
| Clinical Psych | 임상 관련성 | AI 디테일 부족 | 임상 연구 |
| Korean Reviewer | 한글 사용 | 학회 스타일 조정 필요 | 국내 학회 |

💡 팁: 여러 페르소나의 리뷰를 종합하면 더 균형 잡힌 피드백을 얻을 수 있습니다.
""")

---
## 4. 토론

### 토론 질문

1. **AI 리뷰의 품질은 어떤가요?**
   - 실제 리뷰어와 비교해서 어떤 점이 다른가?
   - 어떤 피드백이 유용했고, 어떤 것이 별로였나?

2. **어떤 페르소나가 가장 도움이 되었나요?**
   - 엄격한 리뷰어 vs 건설적 리뷰어
   - 분야 전문가 vs 일반 연구자

3. **AI 리뷰를 논문 작성에 어떻게 활용할 수 있을까?**
   - 투고 전 사전 검토?
   - 특정 섹션(방법론, 통계) 집중 리뷰?
   - 리뷰어 코멘트 대응 연습?

### 워크샵 마무리

오늘 배운 4가지 도구:

| 도구 | 용도 | 커스텀 아이디어 |
|------|------|----------------|
| **Agent Laboratory / PaSa** | 문헌 검색 | PubMed 추가, 심리학 키워드 |
| **LitLLM** | Related Work 생성 | 한글, APA 스타일 |
| **AgentReview** | Peer Review | 지도교수 스타일, 체크리스트 |

### 다음 단계 제안

1. **개인 프로젝트**: 본인 연구 주제로 파이프라인 구축
2. **확장**: 도구들을 연결 (검색 → Related Work → Review)
3. **커스텀**: 연구실 특화 프롬프트/페르소나 개발